In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Concepts
Since I am in a spherically symmetric spacetime, I can fix the angles to arbitrary values and forget them. So I only need to keep track of the radius. The 3D problem is effectively reduced to a 1D one!!! Radial differenciation is then reduced to simply the derivative along the array

In [2]:
class Fields():
    
    def __init__(self, R = 1, N = 11):
        # Domain is a sphere of radius R
        self.R   = R
        self.N   = N
        
        self.dR  = R / N
        self.dTh = 2 * np.pi / N
        self.dPh = np.pi / N
    
        self.r = np.array([(j - 0.5)*self.dR for j in range(1,N+1)])

        self.A_new,  self.A   = np.zeros(N), np.zeros(N)
        self.B_new,  self.B   = np.zeros(N), np.zeros(N)
        self.DA_new, self.DA  = np.zeros(N), np.zeros(N)
        self.DB_new, self.DB  = np.zeros(N), np.zeros(N)
        self.KA_new, self.KA  = np.zeros(N), np.zeros(N)
        self.KB_new, self.KB  = np.zeros(N), np.zeros(N)
        self.Dal_new,self.Dal = np.zeros(N), np.zeros(N)
        self.al_new, self.al  = np.zeros(N), np.zeros(N)
        
    def IC_geodesicSlicing(self, M = 1):
        """
        Geodesic Slicing gauge condition.
                A = B = \psi = 1 + M /(2*r)
                KA = KB = 0
                alpha = 1 = costant
        """
        self.A  = 1 + 0.5 * M / self.r
        self.B  = 1 + 0.5 * M / self.r
        
        for i in range(1,self.N-1):
            self.DA[i] = 0.5 * (np.log(self.A[i+1]) - np.log(self.A[i-1])) / self.dR
            self.DB[i] = 0.5 * (np.log(self.B[i+1]) - np.log(self.B[i-1])) / self.dR
        self.DA[0]  = 0.5 * (- 3 * np.log(self.A[0]) + 4 * np.log(self.A[1]) - np.log(self.A[2])) / self.dR
        self.DA[-1] = 0.5 * (3 * np.log(self.A[-1]) - 4 * np.log(self.A[-2]) + np.log(self.A[-3])) / self.dR
        self.DB[0]  = 0.5 * (- 3 * np.log(self.B[0]) + 4 * np.log(self.B[1]) - np.log(self.B[2])) / self.dR
        self.DB[-1] = 0.5 * (3 * np.log(self.B[-1]) - 4 * np.log(self.B[-2]) + np.log(self.B[-3])) / self.dR
        
        self.DA_Bound = [self.DA[0], self.DA[-1]]
        self.DB_Bound = [self.DB[0], self.DB[-1]]
        
        self.al = np.ones(self.N)
        
    def BC_Dirichlet(self, M = 1):
        self.A[0], self.A[-1] = 1 + 0.5 * M / self.r[0], 1 + 0.5 * M / self.r[-1]
        self.B[0], self.B[-1] = 1 + 0.5 * M / self.r[0], 1 + 0.5 * M / self.r[-1]
        
        self.DA[0], self.DA[-1] = self.DA_Bound
        self.DB[0], self.DB[-1] = self.DB_Bound

In [8]:
def der_r(f, i, dR):
    if i == 0:
        # Forward
        return 0.5 * (- 3 * f[0] + 4 * f[1] - f[2]) / dR
    elif i == len(f)-1:
        # Backward
        return 0.5 * (3 * f[-1] - 4 * f[-2] + f[-3]) / dR
    else:
        # Central
        return 0.5 * (f[i+1] - f[i-1]) / dR

In [17]:
def ev_A(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i] * field.A[i] * field.KA[i]

def ev_B(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i] * field.B[i] * field.KB[i]

def ev_DA(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KA[i] * field.Dal[i]
    p2 = der_r(field.KA, i, field.dR)
    return - 2 * field.al[i] * (p1 + p2)

def ev_DB(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KB[i] * field.Dal[i]
    p2 = der_r(field.KB, i, field.dR)
    return - 2 * field.al[i] * (p1 + p2)


def ev_KA(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    r = field.r[i]
    
    p1 = der_r(field.Dal + field.DB, i, field.dR)
    p2 = field.Dal[i]**2 + 0.5 * (field.Dal[i] * field.DA[i] + field.DB[i]**2 - field.DA[i] * field.DB[i])
    p3 = - field.A[i] * field.KA[i] * (field.KA[i] + 2*field.KB[i])
    p4 = - (field.DA[i] - 2 * field.DB[i]) / r
    return - field.al[i] * (p1 + p2 + p3 + p4) / field.A[i]
    
def ev_KB(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    r = field.r[i]
    
    p1 = der_r(field.DB, i, field.dR)
    p2 = field.Dal[i] * field.DB[i] + field.DB[i]**2 - 0.5 * field.DA[i] * field.DB[i]
    p3 = - (field.DA[i] - 2 * field.Dal[i] - 4 * field.DB[i]) / r
    p4 = - 2 * (field.A[i] - field.B[i]) / (field.B[i] * r**2)
    p5 = field.al[i] * field.KB[i] * (field.KA[i] + 2*field.KB[i])
    return - 0.5 * field.al[i] * (p1+p2+p3+p4) / field.A[i] + p5
    

In [23]:
dt = 0.01
t = 0
N = 100

fields = Fields(R = 1, N = N)
fields.IC_geodesicSlicing()

for _ in range(1):
    t += dt
    for i in range(N):
                #fields.a_symmetryCond()
                fields.BC_Dirichlet()
                
                fields.A_new [i] = fields.A [i] + dt * ev_A (fields, i)
                fields.B_new [i] = fields.B [i] + dt * ev_B (fields, i)
                fields.DA_new[i] = fields.DA[i] + dt * ev_DA(fields, i)
                fields.DB_new[i] = fields.DB[i] + dt * ev_DB(fields, i)
                fields.KA_new[i] = fields.KA[i] + dt * ev_KA(fields, i)
                fields.KB_new[i] = fields.KB[i] + dt * ev_KB(fields, i)
                
                fields.A  = fields.A_new
                fields.B  = fields.B_new
                fields.DA = fields.DA_new
                fields.DB = fields.DB_new
                fields.KA = fields.KA_new
                fields.KB = fields.KB_new

/tmp/ipykernel_165/2415066645.py:45: RuntimeWarning: divide by zero encountered in double_scalars
  return - field.al[i] * (p1 + p2 + p3 + p4) / field.A[i]
/tmp/ipykernel_165/2415066645.py:57: RuntimeWarning: invalid value encountered in double_scalars
  p4 = - 2 * (field.A[i] - field.B[i]) / (field.B[i] * r**2)
/tmp/ipykernel_165/2415066645.py:58: RuntimeWarning: invalid value encountered in double_scalars
  p5 = field.al[i] * field.KB[i] * (field.KA[i] + 2*field.KB[i])
/tmp/ipykernel_165/2415066645.py:42: RuntimeWarning: invalid value encountered in double_scalars
  p2 = field.Dal[i]**2 + 0.5 * (field.Dal[i] * field.DA[i] + field.DB[i]**2 - field.DA[i] * field.DB[i])


In [25]:
fields.A

array([101.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  